# Query Optimization

We'll explore query plans and optimizations for several examples including logical optimizations and examples with and without predicate pushdown.

##### Objectives
1. Logical optimizations
2. Predicate pushdown
3. No predicate pushdown

##### Methods 
- [DataFrame.explain](https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.DataFrame.explain.html#pyspark.sql.DataFrame.explain)


Let’s load our initial DataFrame from CSV (replacing the previous delta format loading). 
The CSV should contain event data with at least a column named **`event_name`**.


In [ ]:
# Replace this path with the actual CSV path uploaded or mounted
csv_path = "/dbfs/tmp/events_data.csv"  # Example location

df = (spark.read
      .option("header", "true")
      .option("inferSchema", "true")
      .csv(csv_path))

display(df)

### Logical Optimization

**`explain(..)`** prints the query plans, optionally formatted by a given explain mode. 
Compare the following logical plan & physical plan, noting how Catalyst handled the multiple **`filter`** transformations.


In [ ]:
from pyspark.sql.functions import col

limit_events_df = (df
                   .filter(col("event_name") != "reviews")
                   .filter(col("event_name") != "checkout")
                   .filter(col("event_name") != "register")
                   .filter(col("event_name") != "email_coupon")
                   .filter(col("event_name") != "cc_info")
                   .filter(col("event_name") != "delivery")
                   .filter(col("event_name") != "shipping_info")
                   .filter(col("event_name") != "press")
                  )

limit_events_df.explain(True)

We could have written the query originally using a single **`filter`** condition ourselves. Compare the previous and following query plans.


In [ ]:
better_df = (df
             .filter((col("event_name").isNotNull()) &
                     (col("event_name") != "reviews") &
                     (col("event_name") != "checkout") &
                     (col("event_name") != "register") &
                     (col("event_name") != "email_coupon") &
                     (col("event_name") != "cc_info") &
                     (col("event_name") != "delivery") &
                     (col("event_name") != "shipping_info") &
                     (col("event_name") != "press"))
            )

better_df.explain(True)

We wouldn't write the following code intentionally, but in a long, complex query you might not notice the duplicate filter conditions. Let's see what Catalyst does with this query.


In [ ]:
stupid_df = (df
             .filter(col("event_name") != "finalize")
             .filter(col("event_name") != "finalize")
             .filter(col("event_name") != "finalize")
             .filter(col("event_name") != "finalize")
             .filter(col("event_name") != "finalize")
            )

stupid_df.explain(True)

### Caching

By default the data of a DataFrame is present on a Spark cluster only while it is being processed during a query -- it is not automatically persisted on the cluster afterwards. 

You can explicitly request Spark to persist a DataFrame on the cluster by invoking its **`cache()`** method.

If you do cache a DataFrame, you should always explicitly evict it from cache by invoking **`unpersist()`** when you no longer need it.

Caching can improve performance in:
- Exploratory data analysis
- Machine learning model training

But caching can degrade performance if:
- It consumes too many cluster resources
- It prevents query optimizations

### Predicate Pushdown

Example reading from a JDBC source, where Catalyst determines that *predicate pushdown* can take place.


In [ ]:
jdbc_url = "jdbc:postgresql://35.89.73.255/training"

# JDBC credentials
conn_properties = {
    "user": "training",
    "password": "training"
}

pp_df = (spark
         .read
         .jdbc(url=jdbc_url,
               table="training.people_1m",
               column="id",
               lowerBound=1,
               upperBound=1000000,
               numPartitions=8,
               properties=conn_properties)
         .filter(col("gender") == "M")
        )

pp_df.explain(True)

Note the lack of a **Filter** and the presence of a **PushedFilters** in the **Scan**. 
The filter operation is pushed to the database and only the matching records are sent to Spark. 
This can greatly reduce the amount of data that Spark needs to ingest.


### No Predicate Pushdown

In comparison, caching the data before filtering eliminates the possibility for the predicate push down.


In [ ]:
cached_df = (spark
            .read
            .jdbc(url=jdbc_url,
                  table="training.people_1m",
                  column="id",
                  lowerBound=1,
                  upperBound=1000000,
                  numPartitions=8,
                  properties=conn_properties
                 )
            )

cached_df.cache()

filtered_df = cached_df.filter(col("gender") == "M")
filtered_df.explain(True)

In this case, Spark has to:
- Read **all** the data from the database
- Cache it
- Perform a filter on cached data

This causes unnecessary data transfer and memory usage.


### Cleanup: Unpersist Cached DataFrame


In [ ]:
cached_df.unpersist()